<a href="https://colab.research.google.com/github/yuki-tamaribuchi/kaggle_codes/blob/master/kaggle_titanic_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!cp drive/'My Drive'/kaggle.json /root/.kaggle/kaggle.json

In [2]:
!kaggle competitions download -c titanic

gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import numpy as np
import pandas as pd
import sklearn

In [4]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

In [5]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
nulls=train_df.isnull().sum()

In [7]:
nulls

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
sex_dummy=pd.get_dummies(train_df['Sex'])
pclass_dummy=pd.get_dummies(train_df['Pclass'],prefix='Pclass')
embarked_dummy=pd.get_dummies(train_df['Embarked'],prefix='Emb')

train_df=train_df.drop(['Ticket','Cabin','Name','Sex','Pclass','Embarked','Cabin'],axis=1)
train_df=pd.concat([train_df,sex_dummy,pclass_dummy,embarked_dummy],axis=1)

train_df['Age']=train_df['Age'].fillna(train_df['Age'].mean())

train_df.head()

y_train=train_df['Survived']
X_train=train_df.drop(['Survived'],axis=1)

In [9]:
nulls=train_df.isnull().sum()

In [10]:
nulls

PassengerId    0
Survived       0
Age            0
SibSp          0
Parch          0
Fare           0
female         0
male           0
Pclass_1       0
Pclass_2       0
Pclass_3       0
Emb_C          0
Emb_Q          0
Emb_S          0
dtype: int64

In [11]:
X_train.head()

,PassengerId,Age,SibSp,Parch,Fare,female,male,Pclass_1,Pclass_2,Pclass_3,Emb_C,Emb_Q,Emb_S
0,1,22.0,1,0,7.2500,0,1,0,0,1,0,0,1
1,2,38.0,1,0,71.2833,1,0,1,0,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,0,0,1,0,0,1
3,4,35.0,1,0,53.1000,1,0,1,0,0,0,0,1
4,5,35.0,0,0,8.0500,0,1,0,0,1,0,0,1


In [12]:
y_train.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [13]:
from sklearn.linear_model import LogisticRegression

In [14]:
log_model=LogisticRegression()
log_model.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [15]:
nulls=test_df.isnull().sum()

In [16]:
nulls

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [17]:
sex_dummy=pd.get_dummies(test_df['Sex'])
pclass_dummy=pd.get_dummies(test_df['Pclass'],prefix='Pclass')
embarked_dummy=pd.get_dummies(test_df['Embarked'],prefix='Emb')

test_df=test_df.drop(['Ticket','Cabin','Name','Sex','Pclass','Embarked','Cabin'],axis=1)
test_df=pd.concat([test_df,sex_dummy,pclass_dummy,embarked_dummy],axis=1)

test_df['Age']=test_df['Age'].fillna(test_df['Age'].mean())
test_df['Fare']=test_df['Fare'].fillna(test_df['Fare'].mean())

test_df.head()

X_test=test_df

In [34]:
y_pred=log_model.predict(X_test)

In [39]:
log_model.score(X_train,y_train)

0.8047138047138047

In [40]:
log_model.coef_

array([[ 1.10649205e-04, -3.69930435e-02, -3.15837335e-01,
        -1.03490023e-01,  2.80096245e-03,  1.65064139e+00,
        -1.02036970e+00,  1.16704233e+00,  2.90670444e-01,
        -8.27441082e-01,  4.21207175e-01,  2.38643544e-01,
        -7.28986678e-02]])

In [41]:
log_model.intercept_

array([0.66650044])

In [42]:
coeff_df=pd.DataFrame([X_train.columns,log_model.coef_[0]]).T

In [43]:
coeff_df

,0,1
0,PassengerId,0.000110649
1,Age,-0.036993
2,SibSp,-0.315837
3,Parch,-0.10349
4,Fare,0.00280096
5,female,1.65064
6,male,-1.02037
7,Pclass_1,1.16704
8,Pclass_2,0.29067
9,Pclass_3,-0.827441


In [58]:
y_pred_df=pd.DataFrame(y_pred)
y_pred_df=y_pred_df.rename(columns={0:'Survived'})

id=pd.DataFrame(X_test['PassengerId'])

In [60]:
result=pd.concat([id,y_pred_df],axis=1)

In [61]:
result

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [64]:
result.to_csv('submission.csv',index=False)

In [65]:
!kaggle competitions submit -c titanic -f submission.csv -m "Titanic Predict with Logistic Regression"

100% 2.77k/2.77k [00:00<00:00, 10.7kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster